In [1]:
import yfinance as yf
import openai
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatPerplexity
import os
import numpy as np

In [ ]:
from dotenv import load_dotenv
 
load_dotenv()
 
os.environ['PPLX_API_KEY'] = os.getenv("openai_api_key")
os.environ['OPENAI_API_KEY'] = os.getenv("perplexity_api_key")

In [21]:
perplexity_chat = ChatPerplexity(
    model="llama-3.1-sonar-small-128k-online",
    temperature=0.1,
)

In [32]:
stock_selection_criteria = {
    "Fundamental Analysis": {
        "Financial Health": {
            "Earnings Growth": {
                "Optimal Range": "5-15% annually",
                "Explanation": "Consistent earnings growth in this range indicates a healthy and growing business."
            },
            "Revenue Growth": {
                "Optimal Range": "5-15% annually",
                "Explanation": "Steady revenue growth suggests a robust business model and market demand."
            },
            "Profit Margins": {
                "Optimal Range": {
                    "Net Profit Margin": "10-20%",
                    "Gross Profit Margin": "30-50%"
                },
                "Explanation": "Higher profit margins indicate better operational efficiency and profitability."
            }
        },
        "Valuation Metrics": {
            "Price-to-Earnings (P/E) Ratio": {
                "Optimal Range": {
                    "Growth Stocks": "15-25",
                    "Value Stocks": "10-20"
                },
                "Explanation": "A P/E ratio within these ranges suggests the stock is reasonably valued relative to its earnings. P/E Ratio below the sector/industry average P/E ratio is also an optimal value."
            },
            "Price-to-Book (P/B) Ratio": {
                "Optimal Range": "Less than 3",
                "Explanation": "A lower P/B ratio may indicate an undervalued stock."
            },
            "Price-to-Sales (P/S) Ratio": {
                "Optimal Range": "Less than 2",
                "Explanation": "Useful for companies with negative earnings; a lower P/S ratio suggests better valuation."
            },
            "Dividend Yield": {
                "Optimal Range": "2-5%",
                "Explanation": "A consistent and growing dividend yield is important for income-oriented investors."
            }
        },
        "Debt Levels": {
            "Debt-to-Equity Ratio": {
                "Optimal Range": "Less than 1",
                "Explanation": "Lower debt levels indicate a financially stable company."
            },
            "Interest Coverage Ratio": {
                "Optimal Range": "Greater than 3",
                "Explanation": "Higher ratios suggest the company can easily meet its interest obligations."
            }
        }
    },
    "Technical Analysis": {
        "Price Trends": {
            "Moving Averages": {
                "Optimal Range": {
                    "50-day Moving Average": "Stock price above the 50-day MA indicates a short-term uptrend.",
                    "200-day Moving Average": "Stock price above the 200-day MA indicates a long-term uptrend."
                },
                "Explanation": "Moving averages help identify trends and potential entry points."
            }
        },
        "Volume": {
            "Trading Volume": {
                "Optimal Range": "Higher than the average daily volume",
                "Explanation": "Higher volume indicates stronger price movements and liquidity."
            }
        },
        "Chart Patterns": {
            "Trend Lines": {
                "Optimal Range": "Identify uptrends with higher highs and higher lows.",
                "Explanation": "Trend lines help visualize the direction of the stock price."
            },
            "Candlestick Patterns": {
                "Optimal Range": "Look for bullish patterns like hammer, engulfing, and morning star.",
                "Explanation": "Bullish patterns suggest potential price increases."
            }
        }
    },
    "Company Fundamentals": {
        "Business Model": {
            "Industry Position": {
                "Optimal Range": "Top 3 in the industry",
                "Explanation": "A strong industry position indicates competitive advantages and market leadership."
            },
            "Growth Prospects": {
                "Optimal Range": "High growth potential in emerging markets or new product lines.",
                "Explanation": "Growth prospects suggest future revenue and earnings growth."
            }
        },
        "Management and Governance": {
            "Management Team": {
                "Optimal Range": "Experienced management with a track record of success.",
                "Explanation": "Strong management is crucial for strategic decision-making and execution."
            },
            "Corporate Governance": {
                "Optimal Range": "Strong governance practices with transparency and accountability.",
                "Explanation": "Good governance reduces the risk of fraud and mismanagement."
            }
        }
    },
    "Market Sentiment": {
        "News and Analyst Reports": {
            "News Sentiment": {
                "Optimal Range": "Positive news sentiment",
                "Explanation": "Positive news indicates favorable market conditions and company performance."
            },
            "Analyst Ratings": {
                "Optimal Range": "Buy or Strong Buy ratings",
                "Explanation": "Favorable analyst ratings suggest potential price appreciation."
            }
        },
        "Social Media and Forums": {
            "Social Media Sentiment": {
                "Optimal Range": "Positive investor sentiment",
                "Explanation": "Positive sentiment on social media can influence market perception and stock price."
            },
            "Investment Forums": {
                "Optimal Range": "Active discussion and positive feedback",
                "Explanation": "Engagement in investment forums can provide insights into market sentiment and potential catalysts."
            }
        }
    },
    "Risk and Volatility": {
        "Beta": {
            "Beta Coefficient": {
                "Optimal Range": {
                    "Defensive Stocks": "Less than 1",
                    "Growth Stocks": "1-1.5"
                },
                "Explanation": "A beta of 1 indicates the stock moves with the market, while a beta greater than 1 indicates higher volatility."
            }
        },
        "Historical Volatility": {
            "Price Fluctuations": {
                "Optimal Range": "Lower historical volatility",
                "Explanation": "Lower volatility suggests more stable price movements."
            }
        }
    },
    "Diversification": {
        "Sector Diversification": {
            "Industry Exposure": {
                "Optimal Range": "Diversified across at least 5-7 industries",
                "Explanation": "Sector diversification mitigates industry-specific risks."
            }
        },
        "Geographic Diversification": {
            "Global Exposure": {
                "Optimal Range": "Investments in companies with operations in multiple regions",
                "Explanation": "Geographic diversification reduces the impact of regional economic downturns."
            }
        }
    },
    "Economic Indicators": {
        "Macroeconomic Factors": {
            "GDP Growth": {
                "Optimal Range": "2-4% annually",
                "Explanation": "Steady GDP growth indicates a healthy economy and potential for business expansion."
            },
            "Inflation Rates": {
                "Optimal Range": "1-3% annually",
                "Explanation": "Moderate inflation rates suggest stable economic conditions."
            }
        },
        "Interest Rates": {
            "Monetary Policy": {
                "Optimal Range": "Stable or slightly increasing interest rates",
                "Explanation": "Stable interest rates indicate a balanced monetary policy and predictable borrowing costs."
            }
        }
    }
}

In [57]:

def get_stock_data(ticker):
    """Fetch recent stock trends using yfinance."""
    stock = yf.Ticker(ticker)
    history = stock.history(period="6mo")  # Last 1 months
    pe_ratio = stock.info.get("trailingPE", "N/A")
    eps = stock.info.get("trailingEps", "N/A")
    
    # Create empty columns for min, max, and average
    lowest_price_weekly = []
    highest_price_weekly = []
    average_price_weekly = []
    average_volume_weekly = []

    # Process data in intervals of size 7
    for i in range(0, len(history), 7):
        subset_low = history['Low'].iloc[i:i+7]  # Get chunk of 7 elements
        lowest_price_weekly.append(subset_low.min())
        subset_high = history['High'].iloc[i:i+7]  # Get chunk of 7 elements
        highest_price_weekly.append(subset_high.max())
        subset_price = history['Close'].iloc[i:i+7]  # Get chunk of 7 elements
        average_price_weekly.append(subset_price.mean())
        subset_volume = history['Volume'].iloc[i:i+7]  # Get chunk of 7 elements
        average_volume_weekly.append(subset_volume.mean())
    return {
        "ticker": ticker,
        "lowest_price_weekly": lowest_price_weekly,
        "highest_price_weekly": highest_price_weekly,
        "average_price_weekly": average_price_weekly,
        "average_volume_weekly": average_volume_weekly,
        "price": history['Close'].iloc[-1],
        "pe_ratio": pe_ratio,
        "eps": eps,
    }

def get_latest_news(ticker):
    """Fetch latest stock news using PerplexityAI."""
    print("Getting latest news of {}".format(ticker))
    search_results = perplexity_chat.predict(f"Provide a summary of details of {ticker} stock with the fields mentioned in the below schema {stock_selection_criteria}.")
    return search_results

In [71]:

def generate_report(stock_data, news):
    print("Generating report of {}".format(stock_data['ticker']))
    """Use OpenAI to generate a stock analysis report."""
    prompt = f"""
    Analyze the 6 month stock {stock_data['ticker']} based on the following:
    - Lowest price weekly: ${stock_data['lowest_price_weekly']}
    - Highest price weekly: ${stock_data['highest_price_weekly']},
    - Average price weekly: ${stock_data['average_price_weekly']},
    - Average price weekly: ${stock_data['average_volume_weekly']},
    - Current Price: ${stock_data['price']},
    - P/E Ratio: {stock_data['pe_ratio']}
    - EPS: {stock_data['eps']}    
    - Summary on stock selection criteria: {news}
    Should an investor buy, hold, or sell this stock?
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

def compare_stocks(stock_details, tickers):
    """Search for comparable stocks based on valuation & industry."""
    print("comparing stocks")
    query = """Provided are details of two stocks - {}. 
            Respond in json format. Which stock is better and buyable under 'better_stock' key and the reason in 'reason' key.
            Stock details: {}
            """.format(tickers, stock_details)
    results = perplexity_chat.predict(query)
    return results

# 🏆 Multi-Agent Execution
def multi_agent_stock_analysis(tickers):
    """Orchestrates the MAS to analyze multiple stocks."""
    stock_details = {}
    comparison = {}
    
    for ticker in tickers:
        stock_data = get_stock_data(ticker)
        news = get_latest_news(ticker)
        report = generate_report(stock_data, news)
        
        stock_details[ticker] = {
            "stock_data": stock_data,
            "news": news,
            "analysis": report,
        }
    comparison = compare_stocks(stock_details, tickers)
    return stock_details, comparison


# Example Run
tickers = ["AAPL", "MSFT"]
stock_details, analysis_results = multi_agent_stock_analysis(tickers)

# for ticker, data in analysis_results.items():
#     print(f"\n📈 {ticker} Analysis Report:\n", data["analysis"])
#     print(f"🔄 Comparable Stock: {data['comparable_stock']}\n")


Getting latest news of AAPL
Generating report of AAPL
Getting latest news of MSFT
Generating report of MSFT
comparing stocks


In [67]:
print(stock_details['AAPL']['analysis'])

Based on the provided analysis, it's suggested that an investor consider their overall investing goals and risk tolerance. Though AAPL stock has demonstrated robust profitability and growth potential, it is currently overvalued judging by its valuation metrics. This indicates there could be a correction in the future, and the stock's price might drop.

The volume of shares traded shows strong liquidity, meaning it is easy for an investor to buy or sell shares without significantly affecting the stock's price. The company's high P/E and P/B ratios suggest that investors are willing to pay a high price for the stock based on the company's earnings and book value, indicating expectation of future growth.

However, the company's debt levels are higher than the optimal range, suggesting potential risk. While the recent news is mostly positive, it is important for investors to continue monitoring the company's financial health, market news and economic conditions to make informed decisions.


In [72]:
print(stock_details['MSFT']['analysis'])

Based on the overall positive analysis and potential for growth, an investor could potentially consider buying or increasing holdings in MSFT stock. Its robust financial health, strong positive outlook in its Cloud and AI segments, reasonable valuation, as well as positive market sentiment suggest that the stock likely holds potential for future growth. 

However, it's important to keep in mind that all investments come with risks. The decision to buy, hold, or sell a stock may primarily depend on an individual's financial goals, risk tolerance, and investing style. Therefore, it is advised to conduct your thorough research or consult with a financial advisor before making any investment decisions.


In [69]:
print(analysis_results)

```json
{
  "better_stock": "MSFT",
  "reason": "Microsoft (MSFT) appears to be well-positioned with strong financial health, favorable market sentiment, and high growth prospects. It has a lower debt-to-equity ratio, higher interest coverage ratio, and a more stable P/E ratio compared to Apple (AAPL). Additionally, MSFT has a higher total return over the past decade and a more consistent dividend yield, indicating better long-term performance and income potential."
}
```

### Explanation

**Microsoft (MSFT) is considered the better stock for several reasons:**

1. **Financial Health**: MSFT has a lower debt-to-equity ratio and a higher interest coverage ratio, indicating better financial stability and the ability to meet its interest obligations[2][5].

2. **Market Sentiment**: The majority of analysts have a "buy" or equivalent rating for MSFT, suggesting potential price appreciation[5].

3. **Growth Prospects**: MSFT has high growth potential in emerging markets and new product line